In [1]:
import csv
import json
import time
import pandas as pd
import numpy as np

import researchpy as rp

import sys  
sys.path.insert(0, 'D:\Google Drive\Analises\Codigos python')
from build_features import *
import ast

from alive_progress import alive_bar
import warnings
warnings.filterwarnings('ignore')

C:\Users\livia\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\livia\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\livia\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
path = 'C:\\Users\\livia\\Dropbox\\HRP Alliance authorship paper\\'

authors = pd.read_csv(path + 'Data 2022-06-15\\authors_gender.csv')
authors.columns
authors.drop("level_0", axis=1, inplace=True)
authors.affiliation = [ast.literal_eval(affiliation) if affiliation != 'Collaborators' else ['Collaborators'] for affiliation in authors.affiliation]
authors.countries = [ast.literal_eval(country.replace(", nan", "").replace("nan, ", "").replace("nan", "")) if "[" in country else [] for country in authors.countries]
authors["countries_name"] = [country[0] if len(set(country)) == 1 else
                               "z No country reported" if len(set(country)) == 0 else
                            "z More than 1 country" for country in authors.countries]

contries_info = pd.read_csv(path + "Reviewed Files\\countries.csv")

contries_info["country"] = [(country).upper() for country in contries_info.name]

contries_info.at[contries_info.country == "Palestine, State of".upper(), "country"] = "PALESTINE"
contries_info.at[contries_info.country == "United Arab Emirates".upper(), "country"] = "ARAB EMIRATES"
contries_info.at[contries_info.country == "Bolivia (Plurinational State of)".upper(), "country"] = "BOLIVIA"
contries_info.at[contries_info.country == "Korea, Republic of".upper(), "country"] = "SOUTH KOREA"
contries_info.at[contries_info.country == "Iran (Islamic Republic of)".upper(), "country"] = "IRAN"
contries_info.at[contries_info.country == "Viet Nam".upper(), "country"] = "VIETNAM"
contries_info.at[contries_info.country == "United Kingdom of Great Britain and Northern Ireland".upper(), "country"] = "UNITED KINGDOM"
contries_info.at[contries_info.country == "Congo, Democratic Republic of the".upper(), "country"] = "REPUBLIC OF THE CONGO"
contries_info.at[contries_info.country == "Congo".upper(), "country"] = "REPUBLIC OF CONGO"
contries_info.at[contries_info.country == "Moldova, Republic of".upper(), "country"] = "REPUBLIC OF MOLDOVA"        
contries_info.at[contries_info.country == "Tanzania, United Republic of".upper(), "country"] = "TANZANIA"  
contries_info.at[contries_info.country == "Russian Federation".upper(), "country"] = 'RUSSIAN FEDERATION'
                 
authors = pd.merge(authors, contries_info, how="left", left_on="countries_name", right_on="country")

who_regions = pd.read_csv(path + "/Reviewed Files/WHO_LMCI_HIC_2.csv")
authors = pd.merge(authors, who_regions, how="left", left_on="alpha-3", right_on="iso3")
authors.columns

Index(['pmid', 'title', 'authors_list', 'citation', 'journal_book',
       'publication_year', 'pmcid', 'doi', 'publication_type', 'mesh_terms',
       'term', 'is_systematic_review', 'abstract', 'count_authors',
       'count_authors_institutions', 'count_authors_individual',
       'count_collaborators', 'count_collaborators_institutions',
       'count_collaborators_individual', 'index', 'full_name', 'affiliation',
       'index_authorship', 'pmid_r', 'is_institutions_name', 'last_name',
       'first_letter_name', 'standard_name', 'deduplicated_name_std',
       'names_variation_std', 'n_variacoes_std', 'countries', 'gender',
       'constant', 'countries_name', 'name', 'alpha-2', 'alpha-3',
       'country-code', 'iso_3166-2', 'region', 'sub-region',
       'intermediate-region', 'region-code', 'sub-region-code',
       'intermediate-region-code', 'country', 'country_name', 'who_area',
       'Unnamed: 2', 'iso3', 'Unnamed: 4'],
      dtype='object')

In [3]:
authors['constant'] = ["constant" for i in authors['gender']]
authors['type_author'] = ["individual" if index_authorship > 0 else 'collaborator' for index_authorship in authors['index_authorship']]
authors['year_of_publication'] = ["1972-1981" if publication_year >= 1972 and publication_year <= 1981 else 
                                    "1982-1991" if publication_year >= 1982 and publication_year <= 1991 else 
                                    "1992-2001" if publication_year >= 1992 and publication_year <= 2001 else 
                                    "2002-2011" if publication_year >= 2002 and publication_year <= 2011 else 
                                    ">2012" for publication_year in authors.publication_year]

authors["affiliation_name"] = ["z More than 1 affiliation" if (len(set(affiliation)) > 1) | ("multiple" in affiliation) else
                               affiliation[0] if len(set(affiliation)) == 1 else
                               "z No affiliation reported" if len(set(affiliation)) == 0 else
                               "z More than 1 affiliation" for affiliation in authors.affiliation]

authors["affiliation_hrp"] = ["HRP" if affiliation == "DEPARTMENT OF REPRODUCTIVE HEALTH AND RESEARCH (RHR) WORLD HEALTH ORGANIZATION (WHO) GENEVA SWITZERLAND." 
                              else "Unknown affiliation" if (affiliation == "z No affiliation reported")
                              else "Non-HRP" for affiliation in authors.affiliation_name]

authors["affiliation_hrp_binary"] = ["HRP" if affiliation == "DEPARTMENT OF REPRODUCTIVE HEALTH AND RESEARCH (RHR) WORLD HEALTH ORGANIZATION (WHO) GENEVA SWITZERLAND." 
                              else "Non-HRP" for affiliation in authors.affiliation_name]

authors["affiliation_hrp_alt"] = ["HRP" if affiliation == "DEPARTMENT OF REPRODUCTIVE HEALTH AND RESEARCH (RHR) WORLD HEALTH ORGANIZATION (WHO) GENEVA SWITZERLAND." 
                              else affiliation if (affiliation == "z No affiliation reported") | (affiliation == "z More than 1 affiliation")
                              else "Other affiliation" for affiliation in authors.affiliation_name]

authors["first_author_affiliation"] = ["Unknown affiliation" for index_authorship in authors['index_authorship']]
authors["first_author_affiliation_alt"] = ["Unknown affiliation" for index_authorship in authors['index_authorship']]
authors["first_author_affiliation_binary"] = ["Non-HRP" for index_authorship in authors['index_authorship']]
authors["last_author_affiliation"] = ["Unknown affiliation" for index_authorship in authors['index_authorship']]
authors["fisrt_author_gender"] = ["unknown" for index_authorship in authors['index_authorship']]
authors["last_author_gender"] = ["unknown" for index_authorship in authors['index_authorship']]
authors["first_author_country"] = ["Unknown country" for index_authorship in authors['index_authorship']]
authors["last_author_country"] = ["Unknown country" for index_authorship in authors['index_authorship']]
authors["first_author_region"] = ["z Unknown region" for index_authorship in authors['index_authorship']]
authors["last_author_region"] = ["z Unknown region" for index_authorship in authors['index_authorship']]
authors["first_author_sub_region"] = ["z Unknown sub-region" for index_authorship in authors['index_authorship']]
authors["first_author_who_area"] = ["z Unknown region" for index_authorship in authors['index_authorship']]
authors["last_author_sub_region"] = ["z Unknown sub-region" for index_authorship in authors['index_authorship']]
authors["last_author_who_area"] = ["z Unknown region" for index_authorship in authors['index_authorship']]
authors["total_female_authors"] = [0 for index_authorship in authors['index_authorship']]
authors["total_male_authors"] = [0 for index_authorship in authors['index_authorship']]
authors["total_unknow_authors"] = [0 for index_authorship in authors['index_authorship']]
authors["total_authors"] = [0 for index_authorship in authors['index_authorship']]
authors["gender_proportion_per_paper"] = [0 for index_authorship in authors['index_authorship']]

list_pmids = list(authors.pmid.unique())

for pmid in list_pmids:
    last_author = authors[(authors.pmid == pmid) & (authors.index_authorship > 1) & 
        (authors.groupby(['pmid'])['index_authorship'].transform(max) == authors['index_authorship'])].reset_index()

    if len(last_author) > 0:
        authors.loc[(authors.pmid == pmid), "last_author_affiliation"] = last_author.iloc[0]["affiliation_hrp"] if not pd.isnull(last_author.iloc[0]["affiliation_hrp"]) else "Unknown affiliation"
        authors.loc[(authors.pmid == pmid), "last_author_affiliation_binary"] = last_author.iloc[0]["affiliation_hrp_binary"] if not pd.isnull(last_author.iloc[0]["affiliation_hrp_binary"]) else "Unknown affiliation"
        authors.loc[(authors.pmid == pmid), "last_author_gender"] = last_author.iloc[0]["gender"] if not pd.isnull(last_author.iloc[0]["gender"]) else "Unknown gender"
        authors.loc[(authors.pmid == pmid), "last_author_country"] = last_author.iloc[0]["countries_name"] if not pd.isnull(last_author.iloc[0]["countries_name"]) else "Unknown country"
        authors.loc[(authors.pmid == pmid), "last_author_sub_region"] = last_author.iloc[0]["sub-region"] if not pd.isnull(last_author.iloc[0]["sub-region"]) else "Unknown sub-region" 
        authors.loc[(authors.pmid == pmid), "last_author_region"] = last_author.iloc[0]["region"] if not pd.isnull(last_author.iloc[0]["region"]) else "z Unknown region"
        authors.loc[(authors.pmid == pmid), "last_author_who_area"] = last_author.iloc[0]["who_area"] if not pd.isnull(last_author.iloc[0]["who_area"]) else "z Unknown region"
        
    
    first_author = authors[(authors.pmid == pmid) & (authors.index_authorship == 1)].reset_index()

    if len(first_author) > 0:
        authors.loc[(authors.pmid == pmid), "first_author_affiliation"] = (first_author.iloc[0]["affiliation_hrp"] 
            if not pd.isnull(first_author.iloc[0]["affiliation_hrp"]) else "Unknown affiliation")
        authors.loc[(authors.pmid == pmid), "first_author_affiliation_binary"] = (first_author.iloc[0]["affiliation_hrp_binary"] 
            if not pd.isnull(first_author.iloc[0]["affiliation_hrp_binary"]) else "Unknown affiliation")
        authors.loc[(authors.pmid == pmid), "first_author_affiliation_alt"] = (first_author.iloc[0]["affiliation_hrp_alt"] 
            if not pd.isnull(first_author.iloc[0]["affiliation_hrp_alt"]) else "Unknown affiliation")
        authors.loc[(authors.pmid == pmid), "first_author_gender"] = (first_author.iloc[0]["gender"] 
            if not pd.isnull(first_author.iloc[0]["gender"]) else "z Unknown region")
        authors.loc[(authors.pmid == pmid), "first_author_country"] = (first_author.iloc[0]["countries_name"]
            if not pd.isnull(first_author.iloc[0]["countries_name"]) else "Unknown country")
        authors.loc[(authors.pmid == pmid), "first_author_sub_region"] = (first_author.iloc[0]["sub-region"] 
            if not pd.isnull(first_author.iloc[0]["sub-region"]) else "z Unknown sub-region")
        authors.loc[(authors.pmid == pmid), "first_author_region"] = (first_author.iloc[0]["region"] 
            if not pd.isnull(first_author.iloc[0]["region"]) else "z Unknown region")
        authors.loc[(authors.pmid == pmid), "first_author_who_area"] = (first_author.iloc[0]["who_area"] 
            if not pd.isnull(first_author.iloc[0]["who_area"]) else "z Unknown region")
        
        
        
    all_authors = authors[(authors.pmid == pmid)]
    total_female  = 0 if "female"  not in all_authors.gender.value_counts().index else all_authors.gender.value_counts()["female"] 
    total_male    = 0 if "male"    not in all_authors.gender.value_counts().index else all_authors.gender.value_counts()["male"] 
    total_unknown = 0 if "unknown" not in all_authors.gender.value_counts().index else all_authors.gender.value_counts()["unknown"] 
    authors.loc[(authors.pmid == pmid), "total_female_authors"] = total_female
    authors.loc[(authors.pmid == pmid), "total_male_authors"] = total_male
    authors.loc[(authors.pmid == pmid), "total_unknow_authors"] = total_unknown
    authors.loc[(authors.pmid == pmid), "gender_proportion_per_paper"] = total_female/total_male if total_male> 0 else total_female
    authors.loc[(authors.pmid == pmid), "total_authors"] = all_authors.gender.value_counts().sum()
    
print("Number of pmids: {}".format(len(authors.pmid.unique())))
print("Number of authors: {}".format(len(authors)))

authors = pd.merge(authors, authors.groupby("deduplicated_name_std", as_index=False)["pmid"].count(),
        on=["deduplicated_name_std"])

authors.rename(columns={"pmid_x": "pmid", "pmid_y": "total_per_papers_authors"}, inplace=True)

authors.head(2)

Number of pmids: 985
Number of authors: 10353


,pmid,title,authors_list,citation,journal_book,publication_year,pmcid,doi,publication_type,mesh_terms,...,last_author_sub_region,last_author_who_area,total_female_authors,total_male_authors,total_unknow_authors,total_authors,gender_proportion_per_paper,last_author_affiliation_binary,first_author_gender,total_per_papers_authors
0,13374,"The W.H.O. expanded programme of research, dev...","A KESSLER, C C STANDLEY","Kessler A, Standley CC. The W.H.O. expanded pr...",Proceedings of the Royal Society of London. Se...,1976,NaN,10.1098/rspb.1976.0103,NaN,"['Female', 'Humans', 'Male', 'Population Contr...",...,Unknown sub-region,z Unknown region,0,1,1,2,0.0,Non-HRP,male,2
1,4650659,The Who Expanded Research Programme in Human R...,"A KESSLER, G W PERKIN, C C STANDLEY","Kessler A, Perkin GW, Standley CC. The Who Exp...",Contraception,1972,NaN,10.1016/0010-7824(72)90034-0,NaN,"['Humans', 'Reproduction', 'Research', 'World ...",...,Unknown sub-region,z Unknown region,0,1,2,3,0.0,Non-HRP,male,2


In [4]:
### Adjusting papers type
path = 'C:\\Users\\livia\\Dropbox\\HRP Alliance authorship paper\\'
papers = pd.read_csv(path + "Reviewed Files\\00_papers_final_VB_v3.csv")
##papers = pd.read_csv(path + "Reviewed Files\\Old\\00_papers_final_VB.csv")
papers.drop("Unnamed: 0", axis = 1, inplace=True)
#papers.drop("Unnamed: 66", axis = 1, inplace=True)
papers.drop_duplicates("pmid", inplace=True)

In [5]:
columns = ['Introductory Journal Article', 'English Abstract', 'Review',
       'Non-P.H.S.', 'Letter', 'Comparative Study', 'Multicenter Study',
       'U.S. Gov\'t', 'Practice Guideline', 'Phase III', 'Dictionary',
       'Comment', 'Research Support', 'Controlled Clinical Trial',
       'Clinical Trial Protocol', 'Historical Article', 'Intramural',
       'Clinical Study', 'Observational Study', 'Published Erratum',
       'Evaluation Study', 'Interview', 'Congress', 'Meta-Analysis',
       'Extramural', 'P.H.S.', 'Phase II', 'Case Reports', 'Editorial',
       'Systematic Review', 'Phase I', 'Non-U.S. Gov\'t', 'Validation Study',
       'Clinical Trial', 'Randomized Controlled Trial', 'N.I.H.',
       'STUDY PROTOCOL', 'INTERVENTION/RCT', 'CASE CONTROL STUDY',
       'CROSS-SECTIONAL', 'COHORT STUDY', 'VALIDATION STUDY',
       'EVALUATION STUDY', 'CASE REPORTS', 'REVIEW', 'ESTIMATES',
       'QUALITATIVE STUDY', 'MIXED-METHODS STUDY', 'CONFERENCE PAPERS',
       'EDITORIAL', 'COMMENT/CORRESPONDENCE', 'STATEMENTS']
       
column_priority = ['STUDY PROTOCOL', 'INTERVENTION/RCT', 'CASE CONTROL STUDY',
       'CROSS-SECTIONAL', 'COHORT STUDY', 'VALIDATION STUDY',
       'EVALUATION STUDY', 'CASE REPORTS', 'REVIEW', 'ESTIMATES',
       'QUALITATIVE STUDY', 'MIXED-METHODS STUDY', 'CONFERENCE PAPERS',
       'EDITORIAL', 'COMMENT/CORRESPONDENCE', 'STATEMENTS']

papers["publication_type_vb"] = [[] for paper in papers.pmid]
papers["publication_type_priority"] = [[] for paper in papers.pmid]

for index, row in papers.iterrows():
    type_vb = []
    type_priority = []
    for column in columns:
        if row[column] == 1:
            type_vb += [column]
            if column in column_priority:
                type_priority += [column]
    papers.at[index, "publication_type_vb"] = type_vb   
    papers.at[index, "publication_type_priority"] = type_priority   

In [6]:
papers_type_reviewes = pd.read_csv(path + "Reviewed Files\\papers_type_VB.csv")[["papers_type", "Suggested new category"]]
papers_type_reviewes["papers_type"] = [item.upper() if not pd.isnull(item) else "" for item in papers_type_reviewes["papers_type"]]
papers_type_reviewes.head(2)

,papers_type,Suggested new category
0,CASE REPORTS,CASE REPORTS
1,CLINICAL STUDY,INTERVENTION/RCT


In [7]:
papers["publication_type_remap"] = papers.publication_type
for index, row in papers.iterrows():
    paper_types = row.publication_type
    paper_types_remap = []
    if not pd.isnull(paper_types):
        itens = paper_types.upper().split(",")
        for item in itens:
            remap = papers_type_reviewes[papers_type_reviewes["papers_type"] == item.strip()].reset_index()
            if len(remap) > 0:
                new_cat = remap.iloc[0]["Suggested new category"].strip()
                if (new_cat not in paper_types_remap and
                    ((len(paper_types_remap) > 0 and new_cat not in ["[RECLASSIFIED]","[RECLASSIFY]"]) or 
                    len(paper_types_remap) == 0)):
                        paper_types_remap += [new_cat]
        if len(paper_types_remap) > 1 and '[RECLASSIFIED]' in paper_types_remap:
            paper_types_remap.remove('[RECLASSIFIED]')
        
        if len(paper_types_remap) > 1 and '[RECLASSIFY]' in paper_types_remap:
            paper_types_remap.remove('[RECLASSIFY]')
        if 'DELETE THIS ARTICLE' in paper_types_remap:
            paper_types_remap.remove('DELETE THIS ARTICLE')
        if len(paper_types_remap) > 1 and 'DELETE CATEGORY' in paper_types_remap:
            paper_types_remap.remove('DELETE CATEGORY')
    paper_types_remap.sort()
    papers.at[index, "publication_type_remap"] = ",".join(paper_types_remap)
papers.publication_type_remap

0       VALIDATION STUDY
1                       
2              EDITORIAL
3                 REVIEW
4                 REVIEW
              ...       
1001        [RECLASSIFY]
1002              REVIEW
1003              REVIEW
1004           EDITORIAL
1005      [RECLASSIFIED]
Name: publication_type_remap, Length: 1006, dtype: object

In [8]:
papers["publication_type_final"] = ["" for paper in papers.pmid]
for index, row in papers.iterrows():
    if len(row.publication_type_priority) > 0:
        papers.at[index, "publication_type_final"] = ",".join(row.publication_type_priority)
    elif "RECLASSIFIED" in row.publication_type_remap:
        #print("teste")
        papers.at[index, "publication_type_final"] = ",".join(row.publication_type_vb)
    else:
        papers.at[index, "publication_type_final"] = ",".join([row.publication_type_remap])

papers["publication_type_final"].value_counts()

REVIEW                    335
CROSS-SECTIONAL           143
COMMENT/CORRESPONDENCE    121
INTERVENTION/RCT          100
EDITORIAL                  54
COHORT STUDY               44
ESTIMATES                  39
QUALITATIVE STUDY          31
EVALUATION STUDY           30
                           26
MIXED-METHODS STUDY        18
STUDY PROTOCOL             13
REMOVE                     12
[RECLASSIFY]               11
CONFERENCE PAPERS          10
CASE CONTROL STUDY          5
CASE REPORTS                5
VALIDATION STUDY            5
STATEMENTS                  4
Name: publication_type_final, dtype: int64

In [9]:
reclassify_types = ['', '[RECLASSIFY]', 
       'Comparative Study,U.S. Gov\'t,Research Support,Non-U.S. Gov\'t',
       'Comparative Study', "INTERVENTION/RCT,CROSS-SECTIONAL"
       'COMMENT/CORRESPONDENCE,EDITORIAL',
       'Introductory Journal Article,Editorial', 
       'Comparative Study,Multicenter Study,U.S. Gov\'t,Research Support,Non-U.S. Gov\'t',
       'Letter,Comparative Study,U.S. Gov\'t,Research Support,Non-U.S. Gov\'t',
       'Comparative Study,Multicenter Study', 'Multicenter Study',
       'Introductory Journal Article,U.S. Gov\'t,Research Support,Non-U.S. Gov\'t']

papers["need_reclassify"] = [True if (pd.isnull(paper_type)) or (paper_type in reclassify_types) else False for paper_type in papers.publication_type_final]
papers["need_reclassify"].value_counts()

False    969
True      37
Name: need_reclassify, dtype: int64

In [10]:
quality_check = papers[~papers["need_reclassify"]]
quality_check = quality_check.sample(frac=0.1)
quality_check.to_csv(path + "quality_check_v2.csv")

In [11]:
papers.to_csv(path + "Reviewed Files\\00_papers_final_VB_v4.csv")
#papers

In [12]:
authors = pd.merge(authors, papers[["pmid", "publication_type_final", "need_reclassify"]], on="pmid", how="left")

In [13]:
tabelaEstiloArtigoCategoricas(authors,'publication_year', 'publication_type_final', probabilidade='col')


publication_year       1972           1973           1974           1976  \
distribuição              N         %    N         %    N         %    N   
publication_type_final                                                     
                          0    (0.0%)    0    (0.0%)    1  (100.0%)    0   
CASE CONTROL STUDY        0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
CASE REPORTS              0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
COHORT STUDY              0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
COMMENT/CORRESPONDENCE    0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
CONFERENCE PAPERS         0    (0.0%)    2  (100.0%)    0    (0.0%)    2   
CROSS-SECTIONAL           0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
EDITORIAL                 0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
ESTIMATES                 0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
EVALUATION STUDY          0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
INTERVENTION/RCT          0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
MIXED-METHODS STUDY       0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
QUALITATIVE STUDY         0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
REVIEW                    0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
STATEMENTS                3  (100.0%)    0    (0.0%)    0    (0.0%)    0   
STUDY PROTOCOL            0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
VALIDATION STUDY          0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
[RECLASSIFY]              0    (0.0%)    0    (0.0%)    0    (0.0%)    0   
All                       3  (100.0%)    2  (100.0%)    1  (100.0%)    2   

publication_year                 1979            ... 2018            2019  \
distribuição                   %    N         %  ...    N         %     N   
publication_type_final                           ...                        
                          (0.0%)    0    (0.0%)  ...   16    (1.7%)    32   
CASE CONTROL STUDY        (0.0%)    0    (0.0%)  ...    0    (0.0%)     0   
CASE REPORTS              (0.0%)    0    (0.0%)  ...   12   (1.27%)     0   
COHORT STUDY              (0.0%)    0    (0.0%)  ...  172  (18.26%)    85   
COMMENT/CORRESPONDENCE    (0.0%)    0    (0.0%)  ...   36   (3.82%)    69   
CONFERENCE PAPERS       (100.0%)    0    (0.0%)  ...    0    (0.0%)     0   
CROSS-SECTIONAL           (0.0%)    0    (0.0%)  ...  134  (14.23%)   134   
EDITORIAL                 (0.0%)    0    (0.0%)  ...   45   (4.78%)    20   
ESTIMATES                 (0.0%)    0    (0.0%)  ...   67   (7.11%)    36   
EVALUATION STUDY          (0.0%)    0    (0.0%)  ...   33    (3.5%)    13   
INTERVENTION/RCT          (0.0%)    0    (0.0%)  ...  112  (11.89%)   261   
MIXED-METHODS STUDY       (0.0%)    0    (0.0%)  ...   16    (1.7%)     6   
QUALITATIVE STUDY         (0.0%)    0    (0.0%)  ...   14   (1.49%)    27   
REVIEW                    (0.0%)    0    (0.0%)  ...  233  (24.73%)   307   
STATEMENTS                (0.0%)    0    (0.0%)  ...    0    (0.0%)     0   
STUDY PROTOCOL            (0.0%)    0    (0.0%)  ...    0    (0.0%)    22   
VALIDATION STUDY          (0.0%)    0    (0.0%)  ...   52   (5.52%)     0   
[RECLASSIFY]              (0.0%)    2  (100.0%)  ...    0    (0.0%)     0   
All                     (100.0%)    2  (100.0%)  ...  942  (100.0%)  1012   

publication_year                 2020            2021             All  \
distribuição                   %    N         %     N         %     N   
publication_type_final                                                  
                         (3.16%)    0    (0.0%)    18   (1.56%)   120   
CASE CONTROL STUDY        (0.0%)    0    (0.0%)     0    (0.0%)    39   
CASE REPORTS              (0.0%)    0    (0.0%)     0    (0.0%)    24   
COHORT STUDY              (8.4%)    7   (0.93%)   204  (17.72%)   740   
COMMENT/CORRESPONDENCE   (6.82%)   84   (11.2%)   115   (9.99%)   700   
CONFERENCE PAPERS         (0.0%)    0    (0.0%) 

In [14]:
authors[authors.publication_type_final == 'INTERVENTION/RCT,CROSS-SECTIONAL'][["need_reclassify", "publication_type_final"]]

,need_reclassify,publication_type_final


In [15]:
authors["need_reclassify"] = [True if (pd.isnull(paper_type)) or (paper_type in reclassify_types) else False for paper_type in authors.publication_type_final]
authors["publication_type_final"] = ["z Need Classification" if (pd.isnull(paper_type)) or (paper_type in reclassify_types) else paper_type for paper_type in authors.publication_type_final]
authors["need_reclassify"].value_counts()

False    8120
True     2233
Name: need_reclassify, dtype: int64

In [16]:
for index, row in authors[authors["need_reclassify"]].iterrows():
    for type_priority in column_priority:
        if (type_priority.lower() in str(row.abstract).lower().replace("-", " ")) or \
            (type_priority.lower() in str(row.title).lower().replace("-", " ")) or \
            (type_priority.lower() in str(row.mesh_terms).lower().replace("-", " ")) or \
            (type_priority.lower() in str(row.abstract).lower()) or \
            (type_priority.lower() in str(row.title).lower()) or \
            (type_priority.lower() in str(row.mesh_terms).lower()):
            authors.at[index, "publication_type_final"] = type_priority

authors["need_reclassify"] = [True if (pd.isnull(paper_type)) or (paper_type in reclassify_types) else False for paper_type in authors.publication_type_final]
authors["publication_type_final"] = ["z Need Classification" if (pd.isnull(paper_type)) or (paper_type in reclassify_types) else paper_type for paper_type in authors.publication_type_final]
authors["need_reclassify"].value_counts()

False    10353
Name: need_reclassify, dtype: int64

In [17]:
first_authors = authors[authors.index_authorship <= 1]
first_authors = first_authors.drop_duplicates("pmid")
print(len(first_authors))
first_authors["need_reclassify"].value_counts().sum()

985


985

In [18]:
authors.to_csv(path + "paper_authors_with_affiliations.csv", index=False)